<a href="https://colab.research.google.com/github/PlatJack/LIDC-IDRI/blob/main/ResourceRecorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm
!pip install kornia
!pip install pylidc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm
import torch.nn.functional as F
from torch import Tensor
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR,StepLR
from torch.utils.data import Subset
import kornia.augmentation as K
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

import matplotlib.pyplot as plt
import pylidc as pl
import math
import glob
from tqdm import tqdm
from timm.models.layers import to_2tuple,trunc_normal_,DropPath,SelectAdaptivePool2d
from timm.models.convnext import LayerNorm2d

import os
import time

import numpy as np
import pandas as pd
import random

In [ ]:
def get_cube_from_img(img3d, center, block_size):
    center_x = center[0]
    center_y = center[1]
    center_z = center[2]

    block_size_x = block_size[0]
    block_size_y = block_size[1]
    block_size_z = block_size[2]

    start_x = max(center_x - block_size_x / 2, 0)
    if start_x + block_size_x > img3d.shape[0]:
        start_x = img3d.shape[0] - block_size_x

    start_y = max(center_y - block_size_y / 2, 0)
    if start_y + block_size_y > img3d.shape[1]:
        start_y = img3d.shape[1] - block_size_y

    start_z = max(center_z - block_size_z / 2, 0)
    if start_z + block_size_z > img3d.shape[2]:
        start_z = img3d.shape[2] - block_size_z

    start_x = int(start_x)
    start_y = int(start_y)
    start_z = int(start_z)
    roi_img3d = img3d[ start_x:start_x + block_size_x,
                      start_y:start_y + block_size_y,
                      start_z:start_z + block_size_z]
    return roi_img3d

data_list = sorted(glob.glob('/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI/*/*/*'))
data_num = len(data_list)
nodule_info = []

block_size = [64,64,56]

save_path_p = 'Data/classification/npyfiles/'
k = 0

print('------------------------------------------------')
for d_idx in range(1, data_num + 1):


    pid = data_list[d_idx].split('/')[-1]
    print('processing-----{}'.format(pid))
    scan = pl.query(pl.Scan).filter(pl.Scan.id == pid).first()
    vol = scan.to_volume()
    print('------------------------------------------------')
    nods = scan.cluster_annotations()
    num_nods = len(nods)

    sid = scan.series_instance_uid

    pixel_info = scan.spacings

    for i, nod_i in enumerate(nods):

        num_name = str(k)
        if k <10 : num_name = '000' + num_name
        elif k <100: num_name = '00' + num_name
        elif k <1000: num_name = '0' + num_name
        save_name = 'Index{}.npy'.format(num_name)

        cent = []
        diameter = 0
        mal_factor = 0
        bbox = []
        num_ann = len(nod_i)
        for j, ann_i in enumerate(nod_i):
            cent.append(ann_i.centroid)
            diameter += ann_i.diameter
            mal_factor += ann_i.feature_vals()[-1]
            bbox.append(ann_i.bbox_dims())
        cent = np.mean(cent,axis=0)
        diameter = diameter/num_ann
        bbox = np.max(bbox,axis=0)
        mal_factor = mal_factor/num_ann


        if mal_factor >3 : mal = 1.
        else: mal = 0.


        nodule_info.append([pid,sid,*pixel_info,*cent,*bbox,diameter,mal_factor,mal])
        patch = get_cube_from_img(vol, cent, block_size)

        if patch.shape[0]<block_size[0]: print('error: index {}'.format(save_name))


        if mal>0: save_path_f = save_path_p + '/True/'
        else: save_path_f = save_path_p + '/False/'

        os.makedirs(save_path_f, exist_ok=True)
        np.save(save_path_f+save_name,patch)
        k +=1

column_index =['patient_id','serisuid',
               'pixel_x','pixel_y','pixel_z',
               'interp_cent_x','interp_cent_y','interp_cent_z',
               'bbox_x','bbox_y','bbox_z',
               'diameter','malignancy_level','malignancy'
               ]
nodule_info_csv = pd.DataFrame(np.array(nodule_info),columns=column_index)
nodule_info_csv = nodule_info_csv.set_index(column_index[0])
nodule_info_csv.to_csv('nodule_info.csv')

------------------------------------------------
processing-----3000923.000000-NA-62357


AttributeError: ignored

In [ ]:
class Dataset(Dataset):
    def __init__(self, path, mode ='train'):
        super().__init__()


        with open('{}/splits/{}.txt'.format(path,mode), 'r') as fin:
            data_list = [line.replace('\n','') for line in fin]


        self.img_path_list = sorted(data_list)
        self.label_list =  [x.split('/')[-2] for x in self.img_path_list]

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        label = int(self.label_list[index]=='True')

        img = np.load(img_path).astype(float)
        img = (img - img.min()) / (img.max() - img.min())
        img = torch.Tensor(img.transpose(2,1,0))


        return img, label


    def __len__(self):
        return len(self.img_path_list)

    def get_labels(self):
        return self.label_list

In [ ]:
class DataAugmentation(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.transforms = nn.Sequential(
            K.RandomVerticalFlip(p=0.5),
            K.RandomThinPlateSpline(p=0.5),
            K.RandomAffine((0,180),p=0.5),
            K.RandomPerspective(0.5,p=0.5),
            )
    @torch.no_grad()
    def forward(self, x: Tensor) -> Tensor:
        x_out = self.transforms(x)
        return x_out

In [ ]:
def train(model, params):
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    optimizer = params['optimizer']
    loss_function=params['loss_function']
    data_path = params['data_path']
    model_path = params['model_path']
    norm = params['norm']
    l_lambda = params['lambda']
    best =0
    ds_tr = Dataset(data_path,'train')
    ds_val = Dataset(data_path,'val')


    dl_tr = DataLoader(ds_tr,
                       batch_size=batch_size,
                       pin_memory=True,
                       shuffle = True,
                       num_workers=0)

    dl_val = DataLoader(ds_val,
                       batch_size=batch_size,
                       pin_memory=True,
                       shuffle = False,
                       num_workers=0)
    dl = {'train':dl_tr, 'val': dl_val}


    augmentation = DataAugmentation()

    scheduler = CosineAnnealingLR(optimizer, T_max=len(dl_tr), eta_min=0, last_epoch=-1)

    for epoch in range(0, num_epochs):
        with tqdm(dl['train'], unit="batch") as tepoch:
            total = 0
            correct = 0

            model.train()
            for data in tepoch:
                tepoch.set_description(f"LR {optimizer.param_groups[0]['lr']},Epoch {epoch}")

                inputs, labels = data
                inputs = augmentation(inputs)
                inputs = inputs.cuda()
                labels = labels.cuda()

                optimizer.zero_grad()

                outputs = model(inputs)

                predictions = outputs.argmax(dim=1, keepdim=True).squeeze()
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
                accuracy = 100*correct / total

                train_loss = loss_function(outputs, labels)

                if norm == 1:

                    l_norm = torch.norm(torch.cat([p.view(-1) for p in model.parameters()]), p=norm)
                    train_loss = train_loss+l_lambda*l_norm
                elif norm ==2:

                    l_norm = torch.norm(torch.cat([p.view(-1) for p in model.parameters()]), p=norm)
                    train_loss = train_loss+l_lambda*l_norm




                train_loss.backward()
                optimizer.step()

                scheduler.step()
                tepoch.set_postfix(loss=train_loss.item(),accuracy=accuracy)


            if (epoch==0) or epoch>15*num_epochs//20:
                total = 0
                correct = 0
                test_loss = 0
                accuracy = []
                loss =[]
                model.eval()
                with torch.no_grad():
                    for i, data in enumerate(dl['val'], 0):
                            inputs, labels = data
                            inputs = inputs.cuda()
                            labels = labels.cuda()

                            outputs = model(inputs)

                            _, predicted = torch.max(outputs, 1)

                            total += labels.size(0)
                            correct += (predicted == labels).sum().item()

                            test_loss = loss_function(outputs, labels)
                            test_loss += test_loss.item()
                accuracy.append(100 * correct/total)
                loss.append(100 *test_loss/total)

                if best<=100*correct/total:

                    save_path = model_path
                    os.makedirs(save_path, exist_ok=True)
                    torch.save(model.state_dict(), save_path + '/trained_model.pt')
                    best = 100*correct/total

                print('Epoch: %d/%d, Tr.loss: %.6f, Val.loss: %.6f, Val.Acc.: %.2f, Best Acc.: %.2f'%(epoch+1, num_epochs, train_loss.item(), 100 *test_loss/total, 100*correct/total,best))
    return best

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self,in_channel, num_features):
        super().__init__()
        self.pool = SelectAdaptivePool2d(1,pool_type='avg')
        self.norm = LayerNorm2d(in_channel)
        self.fc = nn.Linear(in_channel,num_features)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, (nn.Linear)):
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0)


    def forward(self,x):
        x = self.pool(x)
        x = self.norm(x)
        x = torch.flatten(x,1)
        x = self.fc(x)

        return x

In [ ]:
class InputFilter(nn.Module):
    def __init__(self,in_channel, out_channel):
        super().__init__()
        self.upsampling = nn.Upsample(size=(56, 56), mode='bilinear', align_corners=True)
        self.conv = nn.Conv2d(in_channel,out_channel,kernel_size=1)
        self.norm = LayerNorm2d(out_channel)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d)):
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0)


    def forward(self,x):
        x = self.upsampling(x)
        x = self.conv(x)
        x = self.norm(x)

        return x

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()


        model = timm.create_model('convnext_tiny',
                                  pretrained=True,
                                  num_classes=2)

        self.stem = InputFilter(56,96)
        self.stages = model.stages
        self.clf = model.head

    def forward(self, x):

        # input
        x = self.stem(x)
        x = self.stages(x)
        x_out = self.clf(x)


        return x_out

In [ ]:
def set_seed(seed = 0):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_num = 0
set_seed(seed_num)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
loss_function = nn.CrossEntropyLoss().cuda()
model_path = 'model_trained/proposed'

model = Model()
model.cuda()

save_path = model_path
os.makedirs(save_path, exist_ok=True)

optimizer = optim.AdamW(model.parameters(),lr=1e-4, weight_decay=1e-5)

In [ ]:
data_path = 'Data/classification/'
params = {
    'num_epochs': 100,
    'batch_size': 64,
    'seed_num':seed_num,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'data_path': data_path,
    'model_path': model_path,
    'acc_best': 0,
    'norm': 0,
    'lambda':1e-3
    }

train(model, params)

torch.cuda.empty_cache()

FileNotFoundError: ignored